In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# This is Luke's scratchpad notebook.

In [3]:
# SVI doesn't have zip codes, it has fips. So I need to transform FIPS into zip codes, and *then* I can join on the Covid cases onto the data frame

#### Key datasource:
https://www.huduser.gov/portal/datasets/usps_crosswalk.html
Possible note for presentation comments:
- we downloaded the 4th quarter 2018 data Tact-to-zip file, which is the file from HUD that'll translate the data

### Data Prep

In [4]:
bexar_covid = pd.read_csv('Bexar_County_COVID-19_Data_by_Zip_Code.csv')
bexar_covid.columns = bexar_covid.columns.str.lower()
bexar_covid.head(15)

,objectid,zip_code,populationtotals_totpop_cy,positive,casesp100000,zipint,activecases,activecaserate,shape_length,shape_area
0,1,78002,9061,378,4171.724975,78002,24,264.871427,0.427542,0.009546
1,2,78006,5243,96,1831.012779,78006,7,133.511348,0.552725,0.005416
2,4,78015,12254,192,1566.835319,78015,16,130.569610,0.278955,0.002312
3,5,78023,29569,642,2171.192803,78023,49,165.714092,0.886455,0.017922
4,7,78052,699,28,4005.722461,78052,1,143.061517,0.260085,0.001147
5,12,78069,2290,83,3624.454148,78069,5,218.340611,0.254859,0.002041
6,13,78073,7312,300,4102.844639,78073,14,191.466083,0.580798,0.011685
7,14,78101,4485,104,2318.840580,78101,7,156.075808,0.543172,0.006502
8,15,78108,880,7,795.454545,78108,1,113.636364,0.304887,0.000858
9,16,78109,50260,1346,2678.074015,78109,66,131.317151,0.737849,0.007682


In [5]:
bexar_zip = bexar_covid.zip_code.tolist()

In [6]:
bexar_covid.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 74 entries, 0 to 73
Data columns (total 10 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   objectid                    74 non-null     int64  
 1   zip_code                    74 non-null     int64  
 2   populationtotals_totpop_cy  74 non-null     int64  
 3   positive                    74 non-null     int64  
 4   casesp100000                74 non-null     float64
 5   zipint                      74 non-null     int64  
 6   activecases                 74 non-null     int64  
 7   activecaserate              74 non-null     float64
 8   shape_length                74 non-null     float64
 9   shape_area                  74 non-null     float64
dtypes: float64(4), int64(6)
memory usage: 5.9 KB


In [7]:
# Importing the SVI data into it's own DF:

bexar_svi = pd.read_csv('san_antonio_2018_tract.csv')
bexar_svi.columns = bexar_svi.columns.str.lower()
bexar_svi.head()

,st,state,st_abbr,stcnty,county,fips,location,area_sqmi,e_totpop,m_totpop,...,f_crowd,f_noveh,f_groupq,f_theme4,f_total,e_uninsur,m_uninsur,ep_uninsur,mp_uninsur,e_daypop
0,48,TEXAS,TX,48029,Bexar,48029110100,"Census Tract 1101, Bexar County, Texas",1.364296,3102,365,...,0,1,1,3,3,272,130,11.3,5.0,38328
1,48,TEXAS,TX,48029,Bexar,48029110300,"Census Tract 1103, Bexar County, Texas",0.598444,3023,309,...,0,1,0,2,4,606,206,20.0,5.7,9202
2,48,TEXAS,TX,48029,Bexar,48029110500,"Census Tract 1105, Bexar County, Texas",0.448810,2388,243,...,0,1,0,1,9,465,125,19.5,4.4,2044
3,48,TEXAS,TX,48029,Bexar,48029110600,"Census Tract 1106, Bexar County, Texas",0.758136,5301,486,...,0,1,1,3,8,690,265,24.5,7.4,6776
4,48,TEXAS,TX,48029,Bexar,48029110700,"Census Tract 1107, Bexar County, Texas",0.379005,1114,208,...,0,1,0,1,3,251,128,22.5,8.8,3716


In [8]:
bexar_svi.fips.nunique()

366

In [19]:
zips = pd.read_csv('TRACT_ZIP_122018_78s_only.csv')
zips

,tract,zip,res_ratio,bus_ratio,oth_ratio,tot_ratio
0,48007950100,78382,1.00,0.99,1.00,1.00
1,48007950100,78358,0.00,0.01,0.00,0.00
2,48007950200,78382,1.00,1.00,1.00,1.00
3,48007950300,78382,0.63,0.64,0.99,0.63
4,48007950300,78381,0.37,0.36,0.01,0.37
...,...,...,...,...,...,...
2851,48507950200,78872,1.00,1.00,0.00,1.00
2852,48507950301,78839,1.00,1.00,0.94,0.99
2853,48507950301,78829,0.00,0.00,0.00,0.00
2854,48507950301,78834,0.00,0.00,0.06,0.00


In [20]:
zips = zips[zips.zip.isin(bexar_zip)]
zips

,tract,zip,res_ratio,bus_ratio,oth_ratio,tot_ratio
14,48013960100,78264,0.00,0.00,0.00,0.00
16,48013960201,78052,0.63,0.88,0.67,0.64
17,48013960201,78069,0.19,0.07,0.13,0.18
20,48013960201,78002,0.00,0.00,0.00,0.00
22,48013960202,78069,0.19,0.11,0.48,0.20
...,...,...,...,...,...,...
2815,48493000103,78152,0.01,0.01,0.04,0.01
2818,48493000201,78112,0.07,0.00,0.19,0.08
2820,48493000201,78223,0.01,0.00,0.00,0.01
2827,48493000403,78101,0.48,0.14,0.33,0.47


In [28]:
full_df = pd.read_csv('full_san_antonio.csv', index_col = 0)
full_df.head()

,st,state,st_abbr,stcnty,county,tract,location,area_sqmi,e_totpop,m_totpop,...,f_total,e_uninsur,m_uninsur,ep_uninsur,mp_uninsur,e_daypop,zip,population,positive,casesp100000
0,48,TEXAS,TX,48029,Bexar,48029110100,"Census Tract 1101, Bexar County, Texas",1.364296,3102,365,...,3,272,130,11.3,5.0,38328,78101,4485,104,2318.840580
1,48,TEXAS,TX,48029,Bexar,48029110300,"Census Tract 1103, Bexar County, Texas",0.598444,3023,309,...,4,606,206,20.0,5.7,9202,78210,39242,1583,4033.943224
2,48,TEXAS,TX,48029,Bexar,48029110500,"Census Tract 1105, Bexar County, Texas",0.448810,2388,243,...,9,465,125,19.5,4.4,2044,78207,58019,3205,5524.052466
3,48,TEXAS,TX,48029,Bexar,48029110600,"Census Tract 1106, Bexar County, Texas",0.758136,5301,486,...,8,690,265,24.5,7.4,6776,78207,58019,3205,5524.052466
4,48,TEXAS,TX,48029,Bexar,48029110700,"Census Tract 1107, Bexar County, Texas",0.379005,1114,208,...,3,251,128,22.5,8.8,3716,78212,29916,966,3229.041316


In [ ]:
# The one we care about are RPL_Themes and to the right (all the flag columns) Call it 'raw_svi'.

In [49]:
full_df.columns.tolist()

['st',
 'state',
 'st_abbr',
 'stcnty',
 'county',
 'tract',
 'location',
 'area_sqmi',
 'e_totpop',
 'm_totpop',
 'e_hu',
 'm_hu',
 'e_hh',
 'm_hh',
 'e_pov',
 'm_pov',
 'e_unemp',
 'm_unemp',
 'e_pci',
 'm_pci',
 'e_nohsdp',
 'm_nohsdp',
 'e_age65',
 'm_age65',
 'e_age17',
 'm_age17',
 'e_disabl',
 'm_disabl',
 'e_sngpnt',
 'm_sngpnt',
 'e_minrty',
 'm_minrty',
 'e_limeng',
 'm_limeng',
 'e_munit',
 'm_munit',
 'e_mobile',
 'm_mobile',
 'e_crowd',
 'm_crowd',
 'e_noveh',
 'm_noveh',
 'e_groupq',
 'm_groupq',
 'ep_pov',
 'mp_pov',
 'ep_unemp',
 'mp_unemp',
 'ep_pci',
 'mp_pci',
 'ep_nohsdp',
 'mp_nohsdp',
 'ep_age65',
 'mp_age65',
 'ep_age17',
 'mp_age17',
 'ep_disabl',
 'mp_disabl',
 'ep_sngpnt',
 'mp_sngpnt',
 'ep_minrty',
 'mp_minrty',
 'ep_limeng',
 'mp_limeng',
 'ep_munit',
 'mp_munit',
 'ep_mobile',
 'mp_mobile',
 'ep_crowd',
 'mp_crowd',
 'ep_noveh',
 'mp_noveh',
 'ep_groupq',
 'mp_groupq',
 'epl_pov',
 'epl_unemp',
 'epl_pci',
 'epl_nohsdp',
 'spl_theme1',
 'rpl_theme1',
 'epl

In [50]:
columns_to_keep = ['st_abbr',
                    'county',
                    'tract',
                    'area_sqmi',
                    'rpl_themes',
                    'f_pov',
                    'f_unemp',
                    'f_pci',
                    'f_nohsdp',
                    'f_theme1',
                    'f_age65',
                    'f_age17',
                    'f_disabl',
                    'f_sngpnt',
                    'f_theme2',
                    'f_minrty',
                    'f_limeng',
                    'f_theme3',
                    'f_munit',
                    'f_mobile',
                    'f_crowd',
                    'f_noveh',
                    'f_groupq',
                    'f_theme4',
                    'f_total',
                    'zip',
                    'population',
                    'positive',
                    'casesp100000'
                  ]

In [72]:
# Using list comprehension to create a dataframe. Because there are more columns we want to remove than we want to keep, I simply iterated thru the list made above and in essence dropped all columns we didn't want to keep. Easier than using pd.drop.

df = full_df[[c for c in full_df.columns if c in columns_to_keep]]
df.shape, full_df.shape

((366, 29), (366, 128))

In [73]:
df.columns

Index(['st_abbr', 'county', 'tract', 'area_sqmi', 'rpl_themes', 'f_pov',
       'f_unemp', 'f_pci', 'f_nohsdp', 'f_theme1', 'f_age65', 'f_age17',
       'f_disabl', 'f_sngpnt', 'f_theme2', 'f_minrty', 'f_limeng', 'f_theme3',
       'f_munit', 'f_mobile', 'f_crowd', 'f_noveh', 'f_groupq', 'f_theme4',
       'f_total', 'zip', 'population', 'positive', 'casesp100000'],
      dtype='object')

In [74]:
# Renaming the columns:
df.rename(columns = {'rpl_themes': 'raw_svi', 
                     "f_theme1": "f_soci_total", 
                     "f_theme2": "f_comp_total", 
                     "f_theme3": "f_status_total", 
                     "f_theme4": "f_trans_total", 
                     "f_total": "all_flags_total",
                     "f_pov": "f_pov_soci",
                     "f_unemp": "f_unemp_soci", 
                     "f_pci": "f_pci_soci", 
                     "f_nohsdp": "f_nohsdp_soci", 
                     "f_age65": "f_age65_comp", 
                     "f_age17": "f_age17_comp", 
                     "f_disabl": "f_disabl_comp", 
                     "f_sngpnt": "f_sngpnt_comp", 
                     "f_minrty": "f_minrty_status", 
                     "f_limeng": "f_limeng_status", 
                     "f_munit": "f_munit_trans", 
                     "f_mobile": "f_mobile_trans", 
                     "f_crowd": "f_crowd_trans", 
                     "f_noveh": "f_noveh_trans", 
                     "f_groupq": "f_groupq_trans"}, inplace = True)

# Columns should now be in the format that we want: every column name now is attached to it's sub-theme as derived from the CDC's thematic grouping:
# soci == socio-economic theme
# comp == household composition theme
# status == minority status/language theme
# trans == housing type / transportation

df

/Users/luke/opt/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:4304: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,st_abbr,county,tract,area_sqmi,raw_svi,f_pov_soci,f_unemp_soci,f_pci_soci,f_nohsdp_soci,f_soci_total,...,f_mobile_trans,f_crowd_trans,f_noveh_trans,f_groupq_trans,f_trans_total,all_flags_total,zip,population,positive,casesp100000
0,TX,Bexar,48029110100,1.364296,0.6503,0,0,0,0,0,...,0,0,1,1,3,3,78101,4485,104,2318.840580
1,TX,Bexar,48029110300,0.598444,0.6978,1,1,0,0,2,...,0,0,1,0,2,4,78210,39242,1583,4033.943224
2,TX,Bexar,48029110500,0.448810,0.9770,1,1,1,1,4,...,0,0,1,0,1,9,78207,58019,3205,5524.052466
3,TX,Bexar,48029110600,0.758136,0.9841,1,1,1,1,4,...,0,0,1,1,3,8,78207,58019,3205,5524.052466
4,TX,Bexar,48029110700,0.379005,0.9378,0,0,1,0,1,...,0,0,1,0,1,3,78212,29916,966,3229.041316
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
361,TX,Bexar,48029980002,0.601487,-999.0000,-999,-999,-999,-999,-999,...,-999,-999,-999,-999,-999,-999,78226,6269,367,5854.203222
362,TX,Bexar,48029980003,6.233800,0.1477,0,0,0,0,0,...,0,0,0,1,1,1,78236,11007,976,8867.084583
363,TX,Bexar,48029980004,3.779799,-999.0000,-999,-999,-999,-999,-999,...,-999,-999,-999,-999,-999,-999,78202,12654,461,3643.116801
364,TX,Bexar,48029980005,15.269322,-999.0000,0,0,-999,0,-999,...,0,0,0,0,0,-999,78023,29569,642,2171.192803


In [75]:
# Reviewing the new list of columns:

df.columns

Index(['st_abbr', 'county', 'tract', 'area_sqmi', 'raw_svi', 'f_pov_soci',
       'f_unemp_soci', 'f_pci_soci', 'f_nohsdp_soci', 'f_soci_total',
       'f_age65_comp', 'f_age17_comp', 'f_disabl_comp', 'f_sngpnt_comp',
       'f_comp_total', 'f_minrty_status', 'f_limeng_status', 'f_status_total',
       'f_munit_trans', 'f_mobile_trans', 'f_crowd_trans', 'f_noveh_trans',
       'f_groupq_trans', 'f_trans_total', 'all_flags_total', 'zip',
       'population', 'positive', 'casesp100000'],
      dtype='object')

In [84]:
df_pre_row_drop = df.shape
print(df_pre_row_drop)

(366, 29)


In [85]:
# Dropping rows:

# There are 4 rows which are military bases according to the tract information. These rows all return -999 for all the flags and svi score, thus they are not useful for our analysis. Since 4 rows only 1% of our total rows, we opted to simply drop those 4 rows.

df = df[df.raw_svi > 0]